In [ ]:
# 🚀 Clone YOLOv5 repository and install dependencies
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17360, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 17360 (delta 31), reused 12 (delta 9), pack-reused 17311 (from 4)
Receiving objects: 100% (17360/17360), 16.25 MiB | 13.51 MiB/s, done.
Resolving deltas: 100% (11899/11899), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s et

In [ ]:
import os
import zipfile
from google.colab import files

# 📂 Upload dataset ZIP file manually
uploaded = files.upload()

# 🎯 Extract the uploaded ZIP file
zip_file = next(iter(uploaded))  # Get uploaded file name
base_path = "/content/dataset"
os.makedirs(base_path, exist_ok=True)

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(base_path)

print(f"✅ Files extracted to: {base_path}")

# 🔍 Set dataset path (Modify if images are in a different folder)
dataset_path = os.path.join(base_path, "1000")  # Adjust if needed

if os.path.exists(dataset_path):
    print(f"📂 Dataset ready at: {dataset_path}")
else:
    print("❌ Error: Dataset folder not found! Check extraction.")


Saving 1000.zip to 1000.zip
✅ Files extracted to: /content/dataset
📂 Dataset ready at: /content/dataset/1000


In [ ]:
import torch

# 🚀 Load YOLOv5 model (pre-trained on COCO dataset)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# 🎯 Define vehicle classes
VEHICLE_CLASSES = ['car', 'truck', 'bus', 'motorcycle', 'bicycle']

print("✅ YOLOv5 model loaded successfully!")


/usr/local/lib/python3.11/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2025-3-30 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

100%|██████████| 14.1M/14.1M [00:00<00:00, 385MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


✅ YOLOv5 model loaded successfully!


In [ ]:
import pandas as pd
import os
import cv2
from tqdm import tqdm
from PIL import Image

# 📊 Create a CSV file to store results
results_file = os.path.join(dataset_path,"road_traffic_analysis.csv")
results_list = []

# 🖼️ Process each image in the dataset
for image_name in tqdm(os.listdir(dataset_path)):
    image_path = os.path.join(dataset_path, image_name)

    # ✅ Ignore non-image files
    if not os.path.isfile(image_path) or not image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        print(f"⚠️ Skipping {image_name}: Not an image file.")
        continue

    try:
        img = Image.open(image_path)

        # 🔎 Perform inference with YOLOv5
        results = model(img)

        # 📊 Convert results to Pandas DataFrame
        detections = results.pandas().xyxy[0]

        # 🚗 Filter for vehicle classes
        vehicle_detections = detections[detections['name'].isin(VEHICLE_CLASSES)]

        # 🔢 Count total and individual vehicles
        total_vehicles = len(vehicle_detections)
        vehicle_counts = {cls: 0 for cls in VEHICLE_CLASSES}

        for _, row in vehicle_detections.iterrows():
            vehicle_counts[row['name']] += 1

        # 📥 Append results
        results_list.append([
            image_name, total_vehicles,
            vehicle_counts['car'], vehicle_counts['truck'],
            vehicle_counts['bus'], vehicle_counts['motorcycle'], vehicle_counts['bicycle']
        ])

        print(f"✅ Processed {image_name}: {vehicle_counts}")

    except Exception as e:
        print(f"⚠️ Error processing {image_name}: {e}")

# 💾 Save results to CSV
df = pd.DataFrame(results_list, columns=[
    "Image Name", "Total Vehicles", "Cars", "Trucks", "Buses", "Motorcycles", "Bicycles"
])
df.to_csv(results_file, index=False)

print(f"📂 Results saved to {results_file}")


  0%|          | 0/1003 [00:00<?, ?it/s]/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
  0%|          | 1/1003 [00:00<10:06,  1.65it/s]/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache

✅ Processed seq3-drone_0001541.jpg: {'car': 10, 'truck': 4, 'bus': 1, 'motorcycle': 0, 'bicycle': 0}
✅ Processed seq3-drone_0001580.jpg: {'car': 11, 'truck': 3, 'bus': 1, 'motorcycle': 0, 'bicycle': 0}
✅ Processed seq3-drone_0001557.jpg: {'car': 10, 'truck': 2, 'bus': 1, 'motorcycle': 0, 'bicycle': 0}
✅ Processed seq3-drone_0001738.jpg: {'car': 11, 'truck': 5, 'bus': 2, 'motorcycle': 0, 'bicycle': 0}
✅ Processed seq3-drone_0001032.jpg: {'car': 13, 'truck': 3, 'bus': 1, 'motorcycle': 0, 'bicycle': 0}
✅ Processed seq3-drone_0001625.jpg: {'car': 8, 'truck': 2, 'bus': 1, 'motorcycle': 0, 'bicycle': 0}
✅ Processed seq3-drone_0001785.jpg: {'car': 12, 'truck': 3, 'bus': 2, 'motorcycle': 0, 'bicycle': 0}
✅ Processed seq3-drone_0001286.jpg: {'car': 12, 'truck': 1, 'bus': 1, 'motorcycle': 0, 'bicycle': 0}


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
  1%|          | 9/1003 [00:00<01:05, 15.08it/s]/root/.cache/torch/hub/ultralytics_yolov5_master/mod

✅ Processed seq3-drone_0001946.jpg: {'car': 10, 'truck': 9, 'bus': 3, 'motorcycle': 0, 'bicycle': 0}
✅ Processed seq3-drone_0001510.jpg: {'car': 13, 'truck': 3, 'bus': 1, 'motorcycle': 0, 'bicycle': 0}
✅ Processed seq3-drone_0001056.jpg: {'car': 12, 'truck': 2, 'bus': 1, 'motorcycle': 0, 'bicycle': 0}
✅ Processed seq3-drone_0001015.jpg: {'car': 12, 'truck': 3, 'bus': 2, 'motorcycle': 0, 'bicycle': 0}
✅ Processed seq3-drone_0001696.jpg: {'car': 10, 'truck': 3, 'bus': 1, 'motorcycle': 0, 'bicycle': 0}
✅ Processed seq3-drone_0001233.jpg: {'car': 11, 'truck': 1, 'bus': 1, 'motorcycle': 0, 'bicycle': 0}
✅ Processed seq3-drone_0001288.jpg: {'car': 13, 'truck': 2, 'bus': 1, 'motorcycle': 0, 'bicycle': 0}
✅ Processed seq3-drone_0001433.jpg: {'car': 14, 'truck': 1, 'bus': 1, 'motorcycle': 0, 'bicycle': 0}
✅ Processed seq3-drone_0001139.jpg: {'car': 13, 'truck': 1, 'bus': 1, 'motorcycle': 0, 'bicycle': 0}


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
  2%|▏         | 19/1003 [00:01<00:36, 27.01it/s]/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/mo

...

✅ Processed seq3-drone_0001992.jpg: {'car': 9, 'truck': 9, 'bus': 3, 'motorcycle': 0, 'bicycle': 0}
✅ Processed seq3-drone_0001206.jpg: {'car': 11, 'truck': 1, 'bus': 1, 'motorcycle': 0, 'bicycle': 0}
✅ Processed seq3-drone_0001021.jpg: {'car': 11, 'truck': 2, 'bus': 1, 'motorcycle': 1, 'bicycle': 0}
✅ Processed seq3-drone_0001282.jpg: {'car': 11, 'truck': 1, 'bus': 1, 'motorcycle': 0, 'bicycle': 0}
✅ Processed seq3-drone_0001380.jpg: {'car': 14, 'truck': 4, 'bus': 1, 'motorcycle': 0, 'bicycle': 0}
✅ Processed seq3-drone_0001915.jpg: {'car': 11, 'truck': 6, 'bus': 4, 'motorcycle': 0, 'bicycle': 0}
✅ Processed seq3-drone_0001634.jpg: {'car': 10, 'truck': 1, 'bus': 2, 'motorcycle': 0, 'bicycle': 0}
✅ Processed seq3-drone_0001037.jpg: {'car': 12, 'truck': 2, 'bus': 1, 'motorcycle': 0, 'bicycle': 0}


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
 99%|█████████▉| 993/1003 [00:29<00:00, 33.96it/s]/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/m

✅ Processed seq3-drone_0001807.jpg: {'car': 12, 'truck': 3, 'bus': 2, 'motorcycle': 1, 'bicycle': 0}
✅ Processed seq3-drone_0001158.jpg: {'car': 12, 'truck': 0, 'bus': 1, 'motorcycle': 0, 'bicycle': 0}
✅ Processed seq3-drone_0001043.jpg: {'car': 12, 'truck': 2, 'bus': 1, 'motorcycle': 0, 'bicycle': 0}
✅ Processed seq3-drone_0001714.jpg: {'car': 11, 'truck': 2, 'bus': 1, 'motorcycle': 0, 'bicycle': 0}
✅ Processed seq3-drone_0001277.jpg: {'car': 13, 'truck': 1, 'bus': 1, 'motorcycle': 0, 'bicycle': 0}
✅ Processed seq3-drone_0001649.jpg: {'car': 11, 'truck': 3, 'bus': 1, 'motorcycle': 0, 'bicycle': 0}
✅ Processed seq3-drone_0001940.jpg: {'car': 10, 'truck': 6, 'bus': 4, 'motorcycle': 0, 'bicycle': 0}


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
100%|█████████▉| 1001/1003 [00:29<00:00, 33.56it/s]/root/.cache/torch/hub/ultralytics_yolov5_master/

✅ Processed seq3-drone_0001278.jpg: {'car': 11, 'truck': 1, 'bus': 1, 'motorcycle': 0, 'bicycle': 0}
✅ Processed seq3-drone_0001436.jpg: {'car': 13, 'truck': 2, 'bus': 1, 'motorcycle': 0, 'bicycle': 0}
✅ Processed seq3-drone_0001941.jpg: {'car': 11, 'truck': 8, 'bus': 4, 'motorcycle': 0, 'bicycle': 0}
✅ Processed seq3-drone_0001268.jpg: {'car': 12, 'truck': 1, 'bus': 1, 'motorcycle': 0, 'bicycle': 0}
✅ Processed seq3-drone_0001958.jpg: {'car': 12, 'truck': 11, 'bus': 3, 'motorcycle': 0, 'bicycle': 0}
📂 Results saved to /content/dataset/1000/road_traffic_analysis.csv


In [ ]:
# 🛣️ Define function to determine which road a vehicle belongs to
def get_road_section(image_shape, bbox):
    """
    Determines which road a detected vehicle belongs to.
    - image_shape: (height, width) of the image.
    - bbox: Bounding box [x_min, y_min, x_max, y_max] of detected vehicle.
    """
    height, width = image_shape
    mid_x, mid_y = width // 2, height // 2  # Divide image into 4 sections

    x_center = (bbox[0] + bbox[2]) / 2
    y_center = (bbox[1] + bbox[3]) / 2

    if x_center < mid_x and y_center < mid_y:
        return "North Road"
    elif x_center >= mid_x and y_center < mid_y:
        return "East Road"
    elif x_center < mid_x and y_center >= mid_y:
        return "West Road"
    else:
        return "South Road"

# 📊 Create CSV file for road-based vehicle count
road_results_list = []

# 🖼️ Process each image again for road-wise vehicle count
for image_name in tqdm(os.listdir(dataset_path)):
    image_path = os.path.join(dataset_path, image_name)

    if not os.path.isfile(image_path) or not image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue

    try:
        img = Image.open(image_path)
        img_cv2 = cv2.imread(image_path)
        height, width, _ = img_cv2.shape

        results = model(img)
        detections = results.pandas().xyxy[0]
        vehicle_detections = detections[detections['name'].isin(VEHICLE_CLASSES)]

        road_counts = {"North Road": 0, "East Road": 0, "West Road": 0, "South Road": 0}

        for _, row in vehicle_detections.iterrows():
            bbox = [row['xmin'], row['ymin'], row['xmax'], row['ymax']]
            road = get_road_section((height, width), bbox)
            road_counts[road] += 1

        road_results_list.append([image_name, road_counts["North Road"], road_counts["East Road"],
                                  road_counts["West Road"], road_counts["South Road"]])

    except Exception as e:
        print(f"⚠️ Error processing {image_name}: {e}")

df_road = pd.DataFrame(road_results_list, columns=["Image Name", "North Road", "East Road", "West Road", "South Road"])
df_road.to_csv(os.path.join(dataset_path, "road_traffic1.csv"), index=False)

print("📂 Road traffic results saved.")


  0%|          | 0/1004 [00:00<?, ?it/s]/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
  0%|          | 2/1004 [00:00<01:05, 15.37it/s]/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache

📂 Road traffic results saved.


In [ ]:
import os

# 📂 Create folder to save marked images
marked_images_path = os.path.join(dataset_path, "marked_images3")
os.makedirs(marked_images_path, exist_ok=True)

print(f"📂 Marked images will be saved in: {marked_images_path}")


📂 Marked images will be saved in: /content/dataset/1000/marked_images3


In [ ]:
import cv2
import torch
import os
import numpy as np

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Define segmentation lines based on your reference image
def draw_segmentation_lines(img):
    height, width, _ = img.shape
    center_x, center_y = width // 2, height // 2

    # Define segmentation lines
    lines = [
        ((0, center_y), (width, center_y)),  # Horizontal line
        ((center_x, 0), (center_x, height)),  # Vertical line
        ((0, 0), (width, height)),  # Diagonal 1
        ((width, 0), (0, height))  # Diagonal 2
    ]

    for line in lines:
        cv2.line(img, line[0], line[1], (0, 0, 0), 2)

    return img

# Classify vehicle positions based on segmented regions
def classify_vehicle(x, y, width, height):
    center_x, center_y = width // 2, height // 2

    if x < center_x and y < center_y:
        return "North", (0, 255, 0)  # Green
    elif x >= center_x and y < center_y:
        return "East", (255, 0, 0)  # Blue
    elif x < center_x and y >= center_y:
        return "West", (0, 0, 255)  # Red
    else:
        return "South", (255, 255, 0)  # Yellow

# Process images in a folder
def process_images(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith(('.jpg', '.png', '.jpeg')):
            img_path = os.path.join(input_folder, filename)
            img = cv2.imread(img_path)
            if img is None:
                continue

            height, width, _ = img.shape
            img = draw_segmentation_lines(img)  # Draw segmentation

            # Run YOLOv5 inference
            results = model(img)
            detections = results.xyxy[0].cpu().numpy()  # Extract detections

            for det in detections:
                x1, y1, x2, y2, conf, cls = det
                x_center = int((x1 + x2) / 2)
                y_center = int((y1 + y2) / 2)

                road_section, color = classify_vehicle(x_center, y_center, width, height)
                label = f"{road_section} Road"

                # Draw bounding box and label
                cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
                cv2.putText(img, label, (int(x1), int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Save output image
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, img)

# Define input and output folders
input_folder = "/content/dataset/1000"
output_folder = "/content/dataset/1000/marked_images3"

# Process all images
process_images(input_folder, output_folder)

print("Processing complete! Check the 'output_folder' for results.")


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-3-30 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/commo

Processing complete! Check the 'output_folder' for results.


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


In [ ]:
import os
import cv2
import torch
import pandas as pd
from tqdm import tqdm
from PIL import Image

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Vehicle classes in YOLO
VEHICLE_CLASSES = ["car", "bus", "truck", "motorcycle"]

# Define dataset paths
dataset_path = "/content/dataset/1000"
output_base_path = "/content/dataset/1000"

# Ensure output folders exist
road_folders = {
    "North Road": os.path.join(output_base_path, "North_Road"),
    "East Road": os.path.join(output_base_path, "East_Road"),
    "West Road": os.path.join(output_base_path, "West_Road"),
    "South Road": os.path.join(output_base_path, "South_Road"),
}

for folder in road_folders.values():
    os.makedirs(folder, exist_ok=True)

# 🛣️ Define function to determine which road a vehicle belongs to
def get_road_section(image_shape, bbox):
    """
    Determines which road a detected vehicle belongs to.
    - image_shape: (height, width) of the image.
    - bbox: Bounding box [x_min, y_min, x_max, y_max] of detected vehicle.
    """
    height, width = image_shape
    mid_x, mid_y = width // 2, height // 2  # Divide image into 4 sections

    x_center = (bbox[0] + bbox[2]) / 2
    y_center = (bbox[1] + bbox[3]) / 2

    if x_center < mid_x and y_center < mid_y:
        return "North Road"
    elif x_center >= mid_x and y_center < mid_y:
        return "East Road"
    elif x_center < mid_x and y_center >= mid_y:
        return "West Road"
    else:
        return "South Road"

# 📊 Create CSV file for road-based vehicle count
road_results_list = []

# 🖼️ Process each image
for image_name in tqdm(os.listdir(dataset_path)):
    image_path = os.path.join(dataset_path, image_name)

    if not os.path.isfile(image_path) or not image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue

    try:
        img = Image.open(image_path)
        img_cv2 = cv2.imread(image_path)
        height, width, _ = img_cv2.shape

        results = model(img)
        detections = results.pandas().xyxy[0]
        vehicle_detections = detections[detections['name'].isin(VEHICLE_CLASSES)]

        road_counts = {"North Road": 0, "East Road": 0, "West Road": 0, "South Road": 0}

        # Create a copy of the image for visualization
        img_annotated = img_cv2.copy()

        for _, row in vehicle_detections.iterrows():
            bbox = [row['xmin'], row['ymin'], row['xmax'], row['ymax']]
            road = get_road_section((height, width), bbox)
            road_counts[road] += 1

            # Draw bounding box and label
            color = (0, 255, 0) if road == "North Road" else \
                    (255, 0, 0) if road == "East Road" else \
                    (0, 0, 255) if road == "West Road" else \
                    (255, 255, 0)  # Yellow for South

            cv2.rectangle(img_annotated, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), color, 2)
            cv2.putText(img_annotated, road, (int(bbox[0]), int(bbox[1]) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Save annotated image in respective road folder
        for road, count in road_counts.items():
            if count > 0:
                output_path = os.path.join(road_folders[road], image_name)
                cv2.imwrite(output_path, img_annotated)

        # Store counts in CSV data
        road_results_list.append([image_name, road_counts["North Road"], road_counts["East Road"],
                                  road_counts["West Road"], road_counts["South Road"]])

    except Exception as e:
        print(f"⚠️ Error processing {image_name}: {e}")

# Save vehicle count results to CSV
df_road = pd.DataFrame(road_results_list, columns=["Image Name", "North Road", "East Road", "West Road", "South Road"])
df_road.to_csv(os.path.join(output_base_path, "road_traffic.csv"), index=False)

print("✅ Processing complete! Images are saved in respective road folders, and CSV is generated.")


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-3-30 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
  0%|          | 0/1010 [00:00<?, ?it/s]/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
  0%|          | 2/1010 [00:00<01:02, 16.01it/s]/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  wi

✅ Processing complete! Images are saved in respective road folders, and CSV is generated.


In [ ]:
# 🚦 Determine highest traffic road
def get_priority_signal(csv_file):
    df = pd.read_csv(csv_file)
    priority_results = []

    for _, row in df.iterrows():
        image_name = row["Image Name"]
        road_counts = {
            "North Road": row["North Road"],
            "East Road": row["East Road"],
            "West Road": row["West Road"],
            "South Road": row["South Road"],
        }

        max_road = max(road_counts, key=road_counts.get)
        max_count = road_counts[max_road]

        priority_results.append([image_name, max_road, max_count])

        print(f"🚦 Image {image_name}: Highest traffic on {max_road} with {max_count} vehicles")

    df_priority = pd.DataFrame(priority_results, columns=["Image Name", "Highest Traffic Road", "Vehicle Count"])
    df_priority.to_csv(os.path.join(dataset_path, "priority_signals.csv"), index=False)

get_priority_signal(os.path.join(dataset_path, "road_traffic.csv"))

🚦 Image seq3-drone_0001541.jpg: Highest traffic on North Road with 6 vehicles
🚦 Image seq3-drone_0001580.jpg: Highest traffic on West Road with 6 vehicles
🚦 Image seq3-drone_0001557.jpg: Highest traffic on North Road with 6 vehicles
🚦 Image seq3-drone_0001738.jpg: Highest traffic on North Road with 10 vehicles
🚦 Image seq3-drone_0001032.jpg: Highest traffic on North Road with 6 vehicles
🚦 Image seq3-drone_0001625.jpg: Highest traffic on North Road with 5 vehicles
🚦 Image seq3-drone_0001785.jpg: Highest traffic on North Road with 7 vehicles
🚦 Image seq3-drone_0001286.jpg: Highest traffic on North Road with 7 vehicles
🚦 Image seq3-drone_0001946.jpg: Highest traffic on North Road with 13 vehicles
🚦 Image seq3-drone_0001510.jpg: Highest traffic on North Road with 6 vehicles
🚦 Image seq3-drone_0001056.jpg: Highest traffic on North Road with 7 vehicles
🚦 Image seq3-drone_0001015.jpg: Highest traffic on North Road with 6 vehicles
🚦 Image seq3-drone_0001696.jpg: Highest traffic on North Road w